<a href="https://colab.research.google.com/github/BrandaoEid/IBM/blob/master/k_Means_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#k-Means

k-Means é um algoritmo de aprendizado de máquina não supervisionado, ou seja, sem dados rotulados. Em resumo este processo utiliza agrupamentos para segmentar o conjunto de dados baseado na similaridade. 

Este algoritmo faz parte dos algoritmos de **clustering de particionamento**, logo os dados são divididos em sub conjuntos que não se sobrepõe mesmo que estes não sejam rotulados.

<img src="https://i.imgur.com/S65Sk9c.jpg" width="500"/>

Para definir a semelhança são utilizadas funções de distancia, tais como Manhattan, Euclidiana etc. 

É importante saber que não necessariamente a solução encontrada é sempre a ótima. Para isso para escolhermos a quantidade de clusters para um problema é preciso observar a curva entre a média da distância dos pontos em relação a quantidade de clusters. O ponto ideal é onde se encontra o 'cotovelo', pois a partir da quantidade encontrada nessa posição, os ganhos não são significativos e a complexidade aumenta

<img src="https://miro.medium.com/max/1400/1*dChOocbcsLLT1fcxTxj2Ng.png" width="500"/>


Dentre as aplicações do k-Means, temos a segmentação de clientes, que será explorada neste exercício

# Código

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from sklearn.cluster import KMeans

print('Bibliotecas importadas!')

Bibliotecas importadas!


## Aquisição dos dados

In [1]:
!wget -q -O 'customer_segmentation.csv' https://cocl.us/customer_dataset
print('Data downloaded!')

Data downloaded!


In [7]:
customers_df = pd.read_csv('customer_segmentation.csv')
customers_df.head()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,Address,DebtIncomeRatio
0,1,41,2,6,19,0.124,1.073,0.0,NBA001,6.3
1,2,47,1,26,100,4.582,8.218,0.0,NBA021,12.8
2,3,33,2,10,57,6.111,5.802,1.0,NBA013,20.9
3,4,29,2,4,19,0.681,0.516,0.0,NBA009,6.3
4,5,47,1,31,253,9.308,8.908,0.0,NBA008,7.2


## Pré Processamento

Quantidade de linhas e colunas

In [11]:
print('Linhas: {} Colunas: {}'.format(customers_df.shape[0], customers_df.shape[1]))

Linhas: 850 Colunas: 10


Tipo das variáveis

In [10]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Customer Id      850 non-null    int64  
 1   Age              850 non-null    int64  
 2   Edu              850 non-null    int64  
 3   Years Employed   850 non-null    int64  
 4   Income           850 non-null    int64  
 5   Card Debt        850 non-null    float64
 6   Other Debt       850 non-null    float64
 7   Defaulted        700 non-null    float64
 8   Address          850 non-null    object 
 9   DebtIncomeRatio  850 non-null    float64
dtypes: float64(4), int64(5), object(1)
memory usage: 66.5+ KB


Dentre o tipo das variáveis temos apenas um Objeto ("Address"). Como se trata de um atributo categórico e aplicar a distância Euclidiana nesse tipo de variável não traz algo que realmente faça sentido, vou remover esta coluna

In [0]:
df = customers_df.drop(columns=["Address"])

In [13]:
df.head()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,DebtIncomeRatio
0,1,41,2,6,19,0.124,1.073,0.0,6.3
1,2,47,1,26,100,4.582,8.218,0.0,12.8
2,3,33,2,10,57,6.111,5.802,1.0,20.9
3,4,29,2,4,19,0.681,0.516,0.0,6.3
4,5,47,1,31,253,9.308,8.908,0.0,7.2


Em sequencia será feita a normalização dos dados. Esta etapa é importante uma vez que auxilia algoritmos que são baseados em operações matemáticas a interpretar as variáveis de diferentes magnitudes e distribuições da mesma forma

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
X = df.values[:, 1:]
X = np.nan_to_num(X)

In [26]:
cluster_data = StandardScaler().fit_transform(X)
cluster_data

array([[ 0.74291541,  0.31212243, -0.37878978, ..., -0.59048916,
        -0.52379654, -0.57652509],
       [ 1.48949049, -0.76634938,  2.5737211 , ...,  1.51296181,
        -0.52379654,  0.39138677],
       [-0.25251804,  0.31212243,  0.2117124 , ...,  0.80170393,
         1.90913822,  1.59755385],
       ...,
       [-1.24795149,  2.46906604, -1.26454304, ...,  0.03863257,
         1.90913822,  3.45892281],
       [-0.37694723, -0.76634938,  0.50696349, ..., -0.70147601,
        -0.52379654, -1.08281745],
       [ 2.1116364 , -0.76634938,  1.09746566, ...,  0.16463355,
        -0.52379654, -0.2340332 ]])

## Modelo

Utilizarei 3 clusters para segmentar os clientes

In [28]:
num_clusters = 3

#Instanciar o modelo
k_means = KMeans(init= 'k-means++', n_clusters= num_clusters, n_init = 12)

#Ajustar o modelo aos dados
k_means.fit(cluster_data)

#Atribuir as classes que foram assinaladas para cada linha
labels = k_means.labels_

print(labels[0:10])

[1 2 0 1 2 2 1 1 1 2]


## Ajustes

Como temos um cliente para cada linha de dado, e o cluster retornou a qual cluster este cliente pertece, é possível juntar essa informação ao dataset para recuperar alguns insights

In [29]:
df["Labels"] = labels
df.head()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,DebtIncomeRatio,Labels
0,1,41,2,6,19,0.124,1.073,0.0,6.3,1
1,2,47,1,26,100,4.582,8.218,0.0,12.8,2
2,3,33,2,10,57,6.111,5.802,1.0,20.9,0
3,4,29,2,4,19,0.681,0.516,0.0,6.3,1
4,5,47,1,31,253,9.308,8.908,0.0,7.2,2


In [34]:
df.groupby("Labels").median()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,DebtIncomeRatio
Labels,,,,,,,,,
0,439.5,29.5,2.0,3.0,27.0,1.2095,2.3355,1.0,13.2
1,412.0,34.0,1.0,7.0,32.0,0.6060,1.4810,0.0,7.0
2,444.0,43.0,2.0,17.0,83.0,3.1760,7.0360,0.0,13.1


##Conclusões

É possivel notar a partir dos 3 clusters criados as seguintes características
 
* Cluster 0 - Jovens, renda baixa, e com dívidas
* Cluster 1 - Meia idade, renda média e com poucas dívidas
* Cluster 2 - Mais velhos, renda alta e com dívidas





 
